In [ ]:
# ==== DEPENDENCIES ====
# Core data & visualization stack

# Check if key libraries are installed, and install if not
import importlib

def is_module_installed(module_name):
    return importlib.util.find_spec(module_name) is not None

required_libraries = [
    "polars", "pandas", "duckdb", "ipywidgets", "voila",
    "openpyxl", "plotly", "numpy", "gdown", "pyarrow", "jupyterlab","psycopg2-binary","sqlalchemy","tqdm","adbc_driver_postgresql","dotenv"
]

missing_libraries = [lib for lib in required_libraries if not is_module_installed(lib)]

if missing_libraries:
    print(f"Installing missing libraries: {', '.join(missing_libraries)}")
    # Install core libraries
    !pip install {" ".join(missing_libraries)} --quiet
    # Ensure plotly is upgraded
    if "plotly" in missing_libraries:
        !pip install --upgrade plotly
    # Ensure numpy version
    if "numpy" in missing_libraries:
        !pip install numpy>=1.22
else:
    print("All required libraries are already installed.")

# Optional (for profiling, advanced EDA)
# !pip install ydata-profiling --quiet
# Download the parquet file from Google Drive using wget
# The Google Drive link for direct download is structured like this:
# https://drive.google.com/uc?export=download&id=<file_id>
# !wget --no-check-certificate 'https://drive.google.com/file/d/13ovHlFMFH-6XIH-E9YuoqjQRxWl2lfr8/view?usp=sharing' -O ebs.parquet


#disable output
from IPython.display import clear_output
clear_output()

## Displaying the Interactive Report in Voila

To view this notebook as an interactive report with all the widgets and plots rendered, save the notebook and run the following command in your terminal, replacing `your_notebook_name.ipynb` with the actual name of this notebook file:

In [215]:

import pandas as pd
import polars as pl
import plotly.express as px
import plotly.graph_objects as go
import duckdb
import ipywidgets as widgets
from IPython.display import display, HTML
import numpy as np
import os
# ==========================
# ✅ SAFE VOILA INITIAL SETUP
# ==========================
import os
import plotly.io as pio

# ✅ Limit Polars threads if needed
os.environ["POLARS_MAX_THREADS"] = "2"

if os.environ.get("VOILA_APP", None):
    # Running inside Voila
    pio.renderers.default = "iframe"
else:
    # Regular Jupyter or Colab
    pio.renderers.default = "notebook_connected"


print("✅ Voila-safe environment initialized.")


✅ Voila-safe environment initialized.


### Add parquet path

In [ ]:
# ==========================
# ✅ SAFE VOILA INITIAL SETUP — AIVEN DATABASE INTEGRATED (FIXED)
# ==========================
import os
import plotly.io as pio
import warnings
import polars as pl
import pandas as pd
import gdown
from IPython.display import display, clear_output
import sys # Import sys for memory check
from dotenv import load_dotenv
load_dotenv()

warnings.filterwarnings("ignore", category=UserWarning)
# ==================================================
AIVEN_USER = os.getenv("AIVEN_USER", None)
AIVEN_PASSWORD = os.getenv("AIVEN_PASSWORD", None)  # Must be set in environment
AIVEN_HOST = os.getenv("AIVEN_HOST", None)
AIVEN_PORT = os.getenv("AIVEN_PORT", None)
AIVEN_DBNAME = os.getenv("AIVEN_DBNAME", None)
AIVEN_SSLMODE = os.getenv("AIVEN_SSLMODE",None)

# ✅ Construct connection URI
if not AIVEN_PASSWORD:
    raise ValueError("❌ Missing required environment variable: AIVEN_PASSWORD")

AIVEN_CONN_URI = (
    f"postgresql://{AIVEN_USER}:{AIVEN_PASSWORD}"
    f"@{AIVEN_HOST}:{AIVEN_PORT}/{AIVEN_DBNAME}?sslmode={AIVEN_SSLMODE}"
)
AIVEN_TABLE = "employee_benefits"

# ==========================
# 🔹 Local Paths (Fallbacks)
# ==========================
xlsx_path = "content/employee-benefits-in-the-united-states-dataset.xlsx"
parquet_local_path = "content/ebs.parquet"
google_drive_url = "https://drive.google.com/uc?id=1pET8rVOKkR2n8qmbRvsoMiC42ABnGfac"

df_use = None  # final LazyFrame reference

# ==========================
# 🔹 1️⃣ Try Loading Lazily from Aiven PostgreSQL via ADBC
# ==========================
print("🔌 Attempting lazy connection to Aiven PostgreSQL...")

try:
    import adbc_driver_postgresql.dbapi as adbc

    conn = adbc.connect(AIVEN_CONN_URI)
    cursor = conn.cursor()
    cursor.execute(f"SELECT * FROM {AIVEN_TABLE} LIMIT 0")  # schema check

    # ✅ Use Polars read_database with open connection (no engine kwarg)
    # This first call reads only a small sample eagerly to check connection/schema
    df_eager_sample = pl.read_database(f"SELECT * FROM {AIVEN_TABLE} LIMIT 100", conn)
    print(f"✅ Connection established. Table '{AIVEN_TABLE}' detected with {len(df_eager_sample.columns)} columns.")
    print(f"Columns: {list(df_eager_sample.columns)}")

    # --- Lazy proxy setup ---
    # Polars doesn’t yet support lazy SQL scanning directly via ADBC
    # so we emulate lazy behavior by defining a function for downstream code.
    # This function, when called and collected, will execute the query.
    def query_lazy(sql_query: str):
        """Execute a query lazily (returning Polars DataFrame only when collected)."""
        # This re-opens the connection for the specific query execution later
        # A more robust solution might manage a single connection if possible
        return pl.read_database(sql_query, adbc.connect(AIVEN_CONN_URI))


    # --- Define columns to retain ---
    # --- Define columns to retain ---
    cols_needed = [
        "Year",
        "Ownership",
        "Industry",
        "Provision",
        "Estimate",
        "Standard error",
        "Estimate category",
        "Datatype",
        "Characteristic category",
        "Characteristic",
        "Series title",   # no trailing space
        "Series title ",  # with trailing space
        "Series ID",
    ]

    # --- Auto-detect available columns ---
    available_cols = set(df_eager_sample.columns)
    cols_final = [c for c in cols_needed if c in available_cols]

    if not cols_final:
        raise ValueError("❌ None of the expected columns found in Aiven table!")

    # --- Generate SQL-safe list ---
    cols_sql = ", ".join([f'"{c}"' for c in cols_final])

    print(f"✅ Using {len(cols_final)} columns available in Aiven table: {cols_final}")

    # --- Lazy query wrapper ---
    def query_lazy(sql_query: str):
        """Execute a query lazily (returns Polars DataFrame only when collected)."""
        return pl.read_database(sql_query, adbc.connect(AIVEN_CONN_URI))

    # --- Initialize lazy frame ---
    df_use = query_lazy(f"SELECT {cols_sql} FROM {AIVEN_TABLE}")
    print(f"✨ Initialized df_use with {len(cols_final)} columns (Aiven lazy mode).")


    print("\n✨ Initializing df_use as a LAZY frame connected to Aiven.")


except Exception as db_e:
    print(f"⚠️ Could not connect to Aiven PostgreSQL: {db_e}")
    df_use = None


# ==========================
# 🔹 2️⃣ Fallback: Local Parquet
# ==========================
if df_use is None and os.path.exists(parquet_local_path):
    print(f"📦 Loading lazily from local Parquet: {parquet_local_path}")
    try:
        df_use = pl.scan_parquet(parquet_local_path)
        print("✅ Lazy Parquet scan initialized.")
        print(f"Columns: {list(df_use.collect_schema().names())}")
    except Exception as local_err:
        print(f"❌ Error loading local Parquet: {local_err}")
        df_use = None


# ==========================
# 🔹 3️⃣ Fallback: Google Drive Parquet Download
# ==========================
if df_use is None:
    print(f"📥 Attempting download from Google Drive: {google_drive_url}")
    try:
        os.makedirs(os.path.dirname(parquet_local_path), exist_ok=True)
        gdown.download(google_drive_url, parquet_local_path, quiet=False)
        df_use = pl.scan_parquet(parquet_local_path)
        print("✅ Successfully initialized from downloaded Parquet.")
    except Exception as drive_err:
        print(f"❌ Google Drive download failed: {drive_err}")
        df_use = None


# ==========================
# 🔹 4️⃣ Fallback: Excel
# ==========================
if df_use is None and os.path.exists(xlsx_path):
    print(f"📊 Loading from Excel: {xlsx_path}")
    try:
        df_excel = pd.read_excel(xlsx_path, sheet_name="2010-2025")
        df_eager = pl.from_pandas(df_excel)
        print(f"✅ Loaded {df_eager.height:,} rows × {df_eager.width} cols.")
        df_eager.write_parquet(parquet_local_path, compression="snappy")
        df_use = pl.scan_parquet(parquet_local_path)
        print(f"💾 Saved as Parquet at {parquet_local_path}")
    except Exception as excel_err:
        print(f"❌ Error loading Excel: {excel_err}")
        df_use = None


# ==========================
# 🔹 Final Verification and Memory Check
# ==========================
if df_use is not None:
    print("\n🎉 DataFrame 'df_use' initialized successfully.")
    print(f"Type of df_use: {type(df_use)}")

    if isinstance(df_use, pl.DataFrame):
        # Calculate and display memory usage if it's an eager DataFrame
        memory_usage_bytes = sys.getsizeof(df_use)
        memory_usage_mb = memory_usage_bytes / (1024 * 1024)
        print(f"Memory usage of df_use: {memory_usage_mb:.2f} MB")
        try:
             preview = df_use.head(5)
             display(preview)
        except Exception as e:
             print(f"⚠️ Could not preview dataset: {e}")

    elif isinstance(df_use, pl.LazyFrame):
        print("df_use is a LazyFrame and does not hold the entire dataset in memory yet.")
        print("Memory usage will be low until .collect() is called.")
        try:
            # Display schema for LazyFrame
            print("\nSchema of LazyFrame:")
            display(df_use.collect_schema())
        except Exception as e:
            print(f"⚠️ Could not get schema for LazyFrame: {e}")

else:
    print("\n🚨 No dataset could be loaded from any source.")


# clear_output(wait=True) # Keep output visible for memory check
print("\n✅ Initialization complete and Voila-ready.")

🔌 Attempting lazy connection to Aiven PostgreSQL...
✅ Connection established. Table 'employee_benefits' detected with 24 columns.
Columns: ['Survey', 'Estimate category', 'Datatype', 'Provision', 'Ownership', 'Industry', 'Occupation', 'Characteristic category', 'Characteristic', 'Year', 'Estimate', 'Estimate footnote', 'Standard error', 'Standard error footnote', 'Series title ', 'Series ID', 'Survey code', 'Ownership code', 'Estimate code', 'Industry code', 'Occupation code', 'Subcell code', 'Datatype code', 'Provision code']
✅ Using 12 columns available in Aiven table: ['Year', 'Ownership', 'Industry', 'Provision', 'Estimate', 'Standard error', 'Estimate category', 'Datatype', 'Characteristic category', 'Characteristic', 'Series title ', 'Series ID']
✨ Initialized df_use with 12 columns (Aiven lazy mode).

✨ Initializing df_use as a LAZY frame connected to Aiven.

🎉 DataFrame 'df_use' initialized successfully.
Type of df_use: <class 'polars.dataframe.frame.DataFrame'>
Memory usage of

Year,Ownership,Industry,Provision,Estimate,Standard error,Estimate category,Datatype,Characteristic category,Characteristic,Series title,Series ID
i64,str,str,str,str,str,str,str,str,str,str,str
2018,"""Private industry workers""","""All industries""","""Short-term disability: maximum…","""594.00""","""22.23""","""Insurance benefits""","""25th percentile""","""Census area""","""West""","""Weekly maximum benefit dollar …","""NBU21600000000001822275"""
2019,"""Private industry workers""","""All industries""","""Short-term disability: maximum…","""594.00""","""0.45""","""Insurance benefits""","""25th percentile""","""Census area""","""West""","""Weekly maximum benefit dollar …","""NBU21600000000001822275"""
2020,"""Private industry workers""","""All industries""","""Short-term disability: maximum…","""615.00""","""28.28""","""Insurance benefits""","""25th percentile""","""Census area""","""West""","""Weekly maximum benefit dollar …","""NBU21600000000001822275"""
2021,"""Private industry workers""","""All industries""","""Short-term disability: maximum…","""594""","""49.20""","""Insurance benefits""","""25th percentile""","""Census area""","""West""","""Weekly maximum benefit dollar …","""NBU21600000000001822275"""
2022,"""Private industry workers""","""All industries""","""Short-term disability: maximum…","""500""","""45.38""","""Insurance benefits""","""25th percentile""","""Census area""","""West""","""Weekly maximum benefit dollar …","""NBU21600000000001822275"""



✅ Initialization complete and Voila-ready.


In [218]:
import sys
import polars as pl

print(f"Type of df_use: {type(df_use)}")

if isinstance(df_use, pl.DataFrame):
    # Calculate and display memory usage if it's an eager DataFrame
    memory_usage_bytes = sys.getsizeof(df_use)
    memory_usage_mb = memory_usage_bytes / (1024 * 1024)
    print(f"Memory usage of df_use: {memory_usage_mb:.2f} MB")
elif isinstance(df_use, pl.LazyFrame):
    print("df_use is a LazyFrame and does not hold the entire dataset in memory.")
    print("Memory usage will be low until .collect() is called.")
else:
    print("df_use is neither a Polars DataFrame nor a LazyFrame. Cannot determine memory usage in this context.")

Type of df_use: <class 'polars.dataframe.frame.DataFrame'>
Memory usage of df_use: 0.00 MB



## unique counts, nulls, dtypes (Polars)
---



In [219]:
# ==========================
# ✅ Data Summary (Safe for LazyFrame + Streaming)
# ==========================
import polars as pl
from IPython.display import display, clear_output

print("📊 Dataset Summary (Streaming Enabled if Supported)\n")

# Determine frame type
df_type = "LazyFrame" if isinstance(df_use, pl.LazyFrame) else "DataFrame"
print(f"🧠 Frame type detected: {df_type}\n")

# Access schema safely
try:
    schema = (
        df_use.collect_schema().to_dict()  # LazyFrame
        if isinstance(df_use, pl.LazyFrame)
        else {col: df_use[col].dtype for col in df_use.columns}  # DataFrame
    )
except Exception as e:
    print(f"⚠️ Could not get schema: {e}")
    schema = {}

# ✅ Print schema / dtypes
print("🧩 Dtypes:")
for col, dtype in schema.items():
    print(f"  • {col:<30} : {dtype}")
print("\n")

# ✅ Compute null & unique counts efficiently
try:
    print("🧮 Computing null & unique counts... (streaming if possible)\n")

    # Build one set of expressions for both metrics
    summary_exprs = []
    cols = (
        list(df_use.collect_schema().names())
        if isinstance(df_use, pl.LazyFrame)
        else df_use.columns
    )
    for c in cols:
        summary_exprs.append(pl.col(c).is_null().sum().alias(f"{c}_nulls"))
        summary_exprs.append(pl.col(c).n_unique().alias(f"{c}_unique"))

    # ✅ Collect with streaming only if it's a LazyFrame
    if isinstance(df_use, pl.LazyFrame):
        summary = df_use.select(summary_exprs).collect(streaming=True)
    else:
        summary = df_use.select(summary_exprs)

    # ✅ Convert to dict for presentation
    summary_dict = summary.to_dicts()[0]
    null_counts = {
        k.replace("_nulls", ""): int(v)
        for k, v in summary_dict.items()
        if k.endswith("_nulls")
    }
    unique_counts = {
        k.replace("_unique", ""): int(v)
        for k, v in summary_dict.items()
        if k.endswith("_unique")
    }

    # Sort unique counts descending for display
    unique_counts_sorted = dict(
        sorted(unique_counts.items(), key=lambda kv: kv[1], reverse=True)
    )

    print("📉 Null Counts (top 10):")
    display(dict(list(null_counts.items())[:10]))

    print("\n🔢 Unique Counts (top 10):")
    display(dict(list(unique_counts_sorted.items())[:10]))

except Exception as e:
    print(f"❌ Error while computing summary: {e}")

# clear_output(wait=True)
print("✅ Data Summary complete — memory safe and streaming optimized.")


📊 Dataset Summary (Streaming Enabled if Supported)

🧠 Frame type detected: DataFrame

🧩 Dtypes:
  • Year                           : Int64
  • Ownership                      : String
  • Industry                       : String
  • Provision                      : String
  • Estimate                       : String
  • Standard error                 : String
  • Estimate category              : String
  • Datatype                       : String
  • Characteristic category        : String
  • Characteristic                 : String
  • Series title                   : String
  • Series ID                      : String


🧮 Computing null & unique counts... (streaming if possible)

📉 Null Counts (top 10):


{'Year': 0,
 'Ownership': 0,
 'Industry': 0,
 'Provision': 0,
 'Estimate': 0,
 'Standard error': 0,
 'Estimate category': 0,
 'Datatype': 0,
 'Characteristic category': 0,
 'Characteristic': 0}


🔢 Unique Counts (top 10):


{'Series ID': 100124,
 'Series title ': 98086,
 'Estimate': 34209,
 'Standard error': 22597,
 'Provision': 1087,
 'Characteristic': 30,
 'Industry': 29,
 'Year': 16,
 'Datatype': 11,
 'Estimate category': 7}

✅ Data Summary complete — memory safe and streaming optimized.


### Inference :

Owing to high null counts, footnotes must be dropped.

Owing to low variance in data Survey and Survey Code must be removed.

Ownership, Estimate, Industry and Provision code featured can be removed since they have human readable counterparts.

In [ ]:
# ==========================
# ✅ Refine Columns & Clean Data (Streaming + Lazy Safe)
# ==========================
import polars as pl
from IPython.display import display, clear_output

# --- Define columns to retain ---
cols_needed = [
    "Year",
    "Ownership",
    "Industry",
    "Provision",
    "Estimate",
    "Standard error",
    "Estimate category",
    "Datatype",
    "Characteristic category",
    "Characteristic",
    "Series title",   # no trailing space
    "Series title ",  # some old versions
    "Series ID",
]

# --- Snapshot schema (no data load) ---
schema = df_use.schema
available_cols = list(schema.keys())

# ✅ Keep only columns that actually exist in this dataset
cols = [c for c in cols_needed if c in available_cols]

if not cols:
    raise ValueError("🚨 None of the expected analytical columns were found in this dataset!")

# --- Create lazyframe subset ---
df_use = df_use.select(cols)

# ✅ Handle "Series title " vs "Series title" safely
rename_map = {}
if "Series title " in cols and "Series title" not in cols:
    rename_map = {"Series title ": "Series title"}

if rename_map:
    df_use = df_use.rename(rename_map)

# --- Refresh schema after rename ---
schema = df_use.schema
available_cols = list(schema.keys())

# --- Identify string columns safely ---
string_cols = [
    c for c, dtype in schema.items() if dtype == pl.Utf8
]

# --- Clean string columns lazily ---
if string_cols:
    df_use = df_use.with_columns([
        pl.col(c)
        .str.strip_chars()
        .replace("", None)
        .alias(c)
        for c in string_cols
    ])

# --- Safely cast numeric columns (only if they exist) ---
cast_exprs = []
if "Year" in available_cols:
    cast_exprs.append(pl.col("Year").cast(pl.Int32))
if "Estimate" in available_cols:
    cast_exprs.append(pl.col("Estimate").replace({"-": None}).cast(pl.Float64))
if "Standard error" in available_cols:
    cast_exprs.append(pl.col("Standard error").replace({"-": None}).cast(pl.Float64))

if cast_exprs:
    df_use = df_use.with_columns(cast_exprs)

# --- Filter null estimates lazily ---
if "Estimate" in available_cols:
    df_use = df_use.filter(pl.col("Estimate").is_not_null())

# ✅ View quick sample (streaming, tiny eager collect)
print("🧹 Cleaned columns:", list(df_use.schema.keys()))
try:
    sample_preview = df_use.head(5).collect(streaming=True)
    display(sample_preview)
except Exception as e:
    print(f"⚠️ Could not preview cleaned dataset: {e}")

clear_output(wait=True)
print("✅ Data cleaning complete — LazyFrame 'df_use' is ready for analysis.")


✅ Data cleaning complete — LazyFrame 'df_use' is ready for analysis.


In [221]:
# ==========================
# ✅ Validation Checks for LazyFrame (Streaming + DB Safe)
# ==========================
import polars as pl
from IPython.display import display, clear_output

print("🔍 Validating cleaned dataset...\n")

is_lazy = isinstance(df_use, pl.LazyFrame)
is_aiven = 'conn' in globals() and conn is not None and not is_lazy

if is_aiven:
    print("⚙️ Running validation via PostgreSQL (server-side)...")
    try:
        cols = df_use.columns if hasattr(df_use, "columns") else []
        parts = []
        for c in cols:
            parts.append(f"COUNT(*) - COUNT(\"{c}\") AS {c}_nulls")
            parts.append(f"COUNT(DISTINCT \"{c}\") AS {c}_unique")
        sql = f"SELECT {', '.join(parts)} FROM {AIVEN_TABLE}"
        with conn.cursor() as cur:
            cur.execute(sql)
            row = cur.fetchone()
            names = [d[0] for d in cur.description]
            stats = dict(zip(names, row))
        nulls = {k.replace('_nulls', ''): v for k, v in stats.items() if '_nulls' in k}
        uniques = {k.replace('_unique', ''): v for k, v in stats.items() if '_unique' in k}
        print("🧮 Null counts:")
        display(nulls)
        print("\n🔢 Unique counts:")
        display(uniques)
    except Exception as e:
        print(f"❌ Validation failed on DB: {e}")

else:
    try:
        cols = list(df_use.collect_schema().names()) if is_lazy else df_use.columns
        exprs = []
        for c in cols:
            exprs += [pl.col(c).is_null().sum().alias(f"{c}_nulls"),
                      pl.col(c).n_unique().alias(f"{c}_unique")]
        stats = (
            df_use.select(exprs).collect(streaming=True)
            if is_lazy
            else df_use.select(exprs)
        )
        sdict = stats.to_dicts()[0]
        nulls = {k[:-6]: int(v) for k, v in sdict.items() if k.endswith("_nulls")}
        uniques = {k[:-7]: int(v) for k, v in sdict.items() if k.endswith("_unique")}
        print("🧮 Null counts:")
        display(nulls)
        print("\n🔢 Unique counts:")
        display(uniques)
    except Exception as e:
        print(f"❌ Validation error: {e}")

clear_output(wait=True)
print("✅ Validation complete — backend-optimized.")


✅ Validation complete — backend-optimized.


In [222]:
import psutil, os, time

def memory_info(label=""):
    """Display current memory usage info."""
    process = psutil.Process(os.getpid())
    rss = process.memory_info().rss / (1024 ** 2)
    total = psutil.virtual_memory().total / (1024 ** 2)
    used_pct = (rss / total) * 100
    print(f"💾 {label} | Process memory: {rss:.2f} MB ({used_pct:.2f}% of total {total:.0f} MB)")
import psycopg2
import polars as pl
import time, psutil, os

def safe_query(sql: str, conn, verbose=True):
    """Run SQL safely and recover from aborted or lost transactions."""
    start_time = time.perf_counter()
    mem_before = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    for attempt in range(2):
        try:
            # ✅ Attempt rollback if needed
            try:
                conn.rollback()
            except Exception:
                pass

            # ✅ Execute SQL query
            df = pl.read_database(sql, conn)

            mem_after = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)
            elapsed = time.perf_counter() - start_time

            if verbose:
                print(f"✅ SQL executed successfully ({df.height:,} rows).")
                print(f"⏱️ {elapsed:.2f}s | 💾 Δ{mem_after - mem_before:+.2f} MB")

            return df

        except Exception as e:
            print(f"⚠️ Query attempt {attempt + 1} failed: {e}")
            if attempt == 0:
                print("↻ Retrying after rollback and reconnect...")

                # --- Attempt a soft reconnect ---
                try:
                    conn.rollback()
                except Exception:
                    pass

                # optional: refresh connection if closed
                try:
                    import adbc_driver_postgresql.dbapi as adbc
                    if conn is None or hasattr(conn, 'closed') and conn.closed:
                        conn = adbc.connect(AIVEN_CONN_URI)
                        print("🔄 Connection re-established to Aiven.")
                except Exception as e2:
                    print(f"⚠️ Reconnect attempt failed: {e2}")
                continue
            else:
                print("❌ Query failed twice; raising exception.")
                raise e



In [223]:
import polars as pl
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output

# ==================================================
# ✅ Helper: Safe collect (handles both Lazy & DB)
# ==================================================
def ensure_eager(df):
    """Convert LazyFrame → DataFrame safely; passthrough for Aiven."""
    if isinstance(df, pl.LazyFrame):
        return df.collect(engine="streaming")
    return df


# ==================================================
# ✅ 1. Polars-Only or SQL-Aware Filtering
# ==================================================
def polars_filter(
    year=None,
    provision_contains=None,
    ownership=None,
    industries=None,
    estimate_category=None,
    datatype=None,
    case_insensitive=True,
    verbose=True,
):

    start_time = time.perf_counter()
    mem_before = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    is_lazy = isinstance(df_use, pl.LazyFrame)
    is_aiven = 'conn' in globals() and conn is not None and not is_lazy
    print("ISAIVEN",is_aiven)

    def normalize(x):
        if x is None: return None
        if isinstance(x, (list, tuple)): return list(x)
        return [x]

    year = normalize(year)
    ownership = normalize(ownership)
    provision_contains = normalize(provision_contains)
    industries = normalize(industries)
    estimate_category = normalize(estimate_category)
    datatype = normalize(datatype)

    if is_aiven:
      # --- SQL backend ---
      where = ["1=1"]
      if ownership and "All" not in ownership:
          quoted = ", ".join([f"'{o}'" for o in ownership])
          where.append(f"\"Ownership\" IN ({quoted})")
      if year and "All" not in year:
          where.append(f"\"Year\" IN ({', '.join(map(str, year))})")
      if provision_contains and "All" not in provision_contains:
          conds = [f"LOWER(\"Provision\") LIKE '%{p.lower()}%'" for p in provision_contains]
          where.append("(" + " OR ".join(conds) + ")")

      sql = f"""
            SELECT "Year", "Industry", "Estimate", "Provision", "Ownership", "Datatype"
            FROM {AIVEN_TABLE}
            WHERE {' AND '.join(where)}
            """

      df_filtered = safe_query(sql, conn)


      # ✅ Clean invalid values before casting
      df_filtered = df_filtered.with_columns([
          pl.when(pl.col("Estimate").is_in(["-", "", "N/A", "None"]))
            .then(None)
            .otherwise(pl.col("Estimate"))
            .alias("Estimate"),
          pl.col("Year").cast(pl.Int32)
      ])

      # ✅ Safe cast to float (after replacing bad strings)
      df_filtered = df_filtered.with_columns([
          pl.col("Estimate").cast(pl.Float64)
      ])

      # ✅ Drop nulls
      df_filtered = df_filtered.filter(pl.col("Estimate").is_not_null())

      mem_after = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)
      elapsed = time.perf_counter() - start_time

      if verbose:
          print(f"⏱️ Filter runtime: {elapsed:.2f}s")
          print(f"💾 Memory change: {(mem_after - mem_before):.2f} MB")

      if verbose:
          print(f"✅ Retrieved {df_filtered.height:,} rows from Aiven via SQL filter.")
          print(df_filtered.head(5))
      return df_filtered



    # --- Polars path ---
    q = df_use
    if ownership and "All" not in ownership:
        q = q.filter(pl.col("Ownership").is_in(ownership))
    if year and "All" not in year:
        q = q.filter(pl.col("Year").is_in([int(y) for y in year if str(y).isdigit()]))
    if provision_contains and "All" not in provision_contains:
        filters = []
        for p in provision_contains:
            expr = pl.col("Provision")
            if case_insensitive:
                expr = expr.str.to_lowercase()
                p = p.lower()
            filters.append(expr.str.contains(p))
        if filters:
            q = q.filter(pl.any_horizontal(filters))
    q = q.filter(pl.col("Estimate").is_not_null())

    df_filtered = q.collect(engine="streaming")
    if verbose:
        print(f"✅ Filtered rows: {df_filtered.height:,}")
    return df_filtered


# ==================================================
# ✅ 2. Helper for dynamic plot titles
# ==================================================
def make_title(provisions, ownership, year, datatype, estimate_category, industries):
    prov = ", ".join([p for p in provisions if p != "All"]) or "All Provisions"
    own = ", ".join([o for o in ownership if o != "All"]) or "All Ownerships"
    yr = ", ".join([str(y) for y in year if y != "All"]) or "All Years"
    dtype = ", ".join([d for d in datatype if d != "All"]) or "All Datatypes"
    cat = ", ".join([c for c in estimate_category if c != "All"]) or "All Categories"
    ind = ", ".join([i for i in industries if i != "All"]) or "All Industries"
    return f"{prov} — {own} | {cat} | {dtype} | {yr} | {ind}"


# ==================================================
# ✅ 3. Summary Info (Lazy + Streaming-safe)
# ==================================================
df_use_eager = ensure_eager(df_use)

yr_min, yr_max = df_use_eager.select(
    [pl.col("Year").min().alias("min"), pl.col("Year").max().alias("max")]
).row(0)
print("📅 Year range:", yr_min, "→", yr_max)

prov_counts = (
    df_use_eager.group_by("Provision")
    .agg(pl.len().alias("count"))
    .sort("count", descending=True)
)
print("\nTop Provisions:")
display(prov_counts.head(15).to_pandas())

ownerships = (
    df_use_eager.select("Ownership").unique().to_series().drop_nulls().to_list()
)
print("\nOwnership types:", ownerships)


# ==================================================
# ✅ 4. Plotting Functions (Polars-native)
# ==================================================
def plot_industry_trend(df, top_n=15, context=None):
    df = ensure_eager(df)
    if df.is_empty():
        print("⚠️ No data available.")
        return None

    top_inds = (
        df.group_by("Industry")
        .agg(pl.col("Estimate").mean().alias("mean"))
        .sort("mean", descending=True)
        .head(top_n)
        .get_column("Industry")
        .to_list()
    )

    agg = (
        df.filter(pl.col("Industry").is_in(top_inds))
        .group_by(["Year", "Industry"])
        .agg(pl.col("Estimate").mean().alias("Estimate"))
        .sort(["Industry", "Year"])
    )

    fig = px.line(
        agg.to_pandas(),
        x="Year",
        y="Estimate",
        color="Industry",
        markers=True,
        title=make_title(**context) if context else "Industry Trend",
    )
    fig.update_layout(height=600)
    return fig


def plot_cross_section(df, top_n=15, context=None):
    df = ensure_eager(df)
    if df.is_empty():
        print("⚠️ No data to plot.")
        return None

    agg = (
        df.group_by("Industry")
        .agg(pl.col("Estimate").mean().alias("Estimate"))
        .sort("Estimate", descending=True)
        .head(top_n)
    )

    fig = px.bar(
        agg.to_pandas(),
        x="Industry",
        y="Estimate",
        title=make_title(**context) if context else "Cross-section Comparison",
    )
    fig.update_layout(height=600, xaxis_tickangle=-45)
    return fig


def plot_ownership_trend(df, top_n=10, context=None):
    df = ensure_eager(df)
    if df.is_empty():
        print("⚠️ No data available.")
        return None

    agg = (
        df.group_by(["Year", "Ownership"])
        .agg(pl.col("Estimate").mean().alias("Estimate"))
        .sort(["Ownership", "Year"])
    )

    fig = px.line(
        agg.to_pandas(),
        x="Year",
        y="Estimate",
        color="Ownership",
        markers=True,
        title=make_title(**context) if context else "Ownership Trend",
    )
    fig.update_layout(height=600)
    return fig


plot_types = {
    "Industry Trend (Line)": plot_industry_trend,
    "Cross-sectional (Bar)": plot_cross_section,
    "Ownership Trend (Line)": plot_ownership_trend,
}


# ==================================================
# ✅ 5. Widget Setup (Safe + Formatted)
# ==================================================
ownership_options = ["All"] + sorted(
    df_use_eager.select("Ownership").unique().to_series().drop_nulls().to_list()
)
provision_options = ["All"] + sorted(
    df_use_eager.select("Provision").unique().to_series().drop_nulls().to_list()
)
year_options = ["All"] + sorted(
    [str(y) for y in df_use_eager.select("Year").unique().to_series().drop_nulls().to_list()]
)
industries = ["All"] + sorted(
    df_use_eager.select("Industry").unique().to_series().drop_nulls().to_list()
)
estimate_categories = ["All"] + sorted(
    df_use_eager.select("Estimate category").unique().to_series().drop_nulls().to_list()
)
datatypes = ["All"] + sorted(
    df_use_eager.select("Datatype").unique().to_series().drop_nulls().to_list()
)

# Display summary in readable format
print("\nEstimate Categories:")
print("\n".join(["All"] + estimate_categories))
print("\nDatatypes:")
print("\n".join(["All"] + datatypes))

# --- Define interactive widgets ---
w_plot_type = widgets.Dropdown(options=list(plot_types.keys()), description="Plot Type:")
w_provision = widgets.SelectMultiple(options=provision_options, value=("All",), description="Provision(s):")
w_ownership = widgets.SelectMultiple(options=ownership_options, value=("Private industry workers",), description="Ownership(s):")
w_year = widgets.SelectMultiple(options=year_options, value=("All",), description="Year(s):")
w_topn = widgets.IntSlider(value=10, min=5, max=50, step=5, description="Top N:")
w_industry = widgets.SelectMultiple(options=industries, value=("All",), description="Industry:")
w_estimate_cat = widgets.SelectMultiple(options=estimate_categories, value=("All",), description="Estimate Category:")
w_datatype = widgets.SelectMultiple(options=datatypes, value=("All",), description="Datatype:")

out = widgets.Output()


📅 Year range: 2010 → 2025

Top Provisions:


,Provision,count
0,Access to all retirement benefits,2746
1,Participation rate for all retirement benefits,2746
2,Take-up rate for all retirement benefits,2746
3,Medical care: family coverage employee share o...,2745
4,Access to healthcare benefits,2745
5,Take-up rate for outpatient prescription drugs...,2745
6,Access to outpatient prescription drugs benefits,2745
7,Participation rate for outpatient prescription...,2745
8,Access to paid jury duty leave,2745
9,Access to both medical care and retirement ben...,2745



Ownership types: ['Private industry workers', 'State government workers', 'State and local government workers', 'Local government workers', 'Civilian workers']

Estimate Categories:
All
All
Benefit combinations
Financial benefits
Healthcare benefits
Insurance benefits
Leave benefits
Quality of life benefits
Retirement benefits

Datatypes:
All
All
10th percentile
25th percentile
50th percentile - median
75th percentile
90th percentile
Access rate
Average
Establishments offering benefits
Participation rate
Share of premiums
Take-up rate


In [224]:

# ==================================================
# ✅ 6. Interactive Callback
# ==================================================
import plotly.io as pio

def update_plot(year, industries, provision, ownership, estimate_category, datatype, plot_type, top_n):
    with out:
        clear_output(wait=True)
        print("🔄 Updating plot...")

        context = {
            "provisions": provision,
            "ownership": ownership,
            "year": year,
            "estimate_category": estimate_category,
            "datatype": datatype,
            "industries": industries,
        }

        df_filtered = polars_filter(
            year=year,
            industries=industries,
            provision_contains=provision,
            ownership=ownership,
            estimate_category=estimate_category,
            datatype=datatype,
        )
        if df_filtered is None:
            print("⚠️ No matching data.")
            return

        fig = plot_types[plot_type](df_filtered, top_n=top_n, context=context)
        if fig:
            fig.show()

widgets.interactive_output(
    update_plot,
    {
        "year": w_year,
        "industries": w_industry,
        "provision": w_provision,
        "ownership": w_ownership,
        "estimate_category": w_estimate_cat,
        "datatype": w_datatype,
        "plot_type": w_plot_type,
        "top_n": w_topn,
    },
)

display(
    widgets.VBox(
        [
            w_plot_type,
            widgets.HBox([w_ownership, w_year, w_topn]),
            w_industry,
            w_provision,
            widgets.HBox([w_estimate_cat, w_datatype]),
            out,
        ]
    )
)


# Medical cost Analysis

In [225]:
# ==================================================
# ✅ MEDICAL COST DASHBOARD — Aiven + Lazy Hybrid
# ==================================================
import polars as pl
import plotly.express as px
import textwrap
import ipywidgets as widgets
from IPython.display import display, clear_output
import psutil, time, os


# --- Safe collector for streaming mode ---
def collect_streaming_safe(lf: pl.LazyFrame, streaming: bool = True) -> pl.DataFrame:
    if not streaming:
        return lf.collect()
    try:
        return lf.collect(engine="streaming")
    except TypeError:
        return lf.collect(streaming=True)


# --- Detect backend ---
is_aiven = "conn" in globals() and conn is not None
print(f"🔍 Backend mode: {'Aiven PostgreSQL' if is_aiven else 'Local Parquet/LazyFrame'}")


# ==================================================
# ✅ STEP 1: Build base data source
# ==================================================
if not is_aiven:
    # Local Polars (Lazy) path
    filtered_for_medical_cost = df_use.filter(
        (pl.col("Datatype") == "Average") &
        (pl.col("Provision").str.contains("Medical care:", literal=True))
    )

    unique_medical_provisions = (
        filtered_for_medical_cost
        .select("Provision")
        .unique()
        .collect()
        .to_series()
        .drop_nulls()
        .to_list()
    )

    ownership_options = (
        df_use
        .select("Ownership")
        .unique()
        .collect()
        .to_series()
        .drop_nulls()
        .to_list()
    )

    available_years = (
        filtered_for_medical_cost
        .select("Year")
        .unique()
        .collect()
        .to_series()
        .drop_nulls()
        .to_list()
    )

else:
    # --- Aiven (SQL) mode ---
    print("📡 Fetching metadata from Aiven (light queries)...")

    # Small, metadata-only queries — super fast
    ownership_options = (
        pl.read_database('SELECT DISTINCT "Ownership" FROM employee_benefits', conn)
        .to_series().drop_nulls().to_list()
    )

    unique_medical_provisions = (
        pl.read_database(
            """SELECT DISTINCT "Provision"
               FROM employee_benefits
               WHERE "Datatype" = 'Average' AND "Provision" ILIKE 'Medical care:%'""",
            conn
        )
        .to_series().drop_nulls().to_list()
    )

    available_years = (
        pl.read_database('SELECT DISTINCT "Year" FROM employee_benefits', conn)
        .to_series().drop_nulls().to_list()
    )

# --- Clean defaults ---
selected_ownership = ownership_options[0] if ownership_options else None
selected_provision = unique_medical_provisions[0] if unique_medical_provisions else None

available_years = sorted([int(y) for y in available_years if isinstance(y, (int, float))])
min_year = available_years[0] if available_years else 2010
max_year = available_years[-1] if available_years else 2025


# ==================================================
# ✅ STEP 2: Define Widgets
# ==================================================
w_year_slider = widgets.IntSlider(
    value=max_year, min=min_year, max=max_year, step=1,
    description="Year (Up to):", continuous_update=False
)
w_ownership_dropdown = widgets.Dropdown(
    options=ownership_options, value=selected_ownership, description="Ownership:"
)
w_provision_dropdown = widgets.Dropdown(
    options=unique_medical_provisions, value=selected_provision,
    description="Provision:", layout=widgets.Layout(width="600px")
)
output_widget = widgets.Output()


# ==================================================
# ✅ STEP 3: Reactive Plot Function
# ==================================================
def update_plot(year, ownership, provision):
    with output_widget:
        clear_output(wait=True)
        print(f"📊 Filtering for Year ≤ {year}, Ownership: {ownership}, Provision: {provision}")

        start_time = time.perf_counter()
        mem_before = psutil.Process(os.getpid()).memory_info().rss / (1024**2)

        if is_aiven:
            sql = f"""
            SELECT "Year", "Industry", "Estimate"
            FROM employee_benefits
            WHERE "Datatype" = 'Average'
              AND "Ownership" = '{ownership}'
              AND "Provision" = '{provision}'
              AND "Year" <= {year}
              AND "Estimate" NOT IN ('-', '', 'N/A')
            """
            df_filtered = safe_query(sql, conn, verbose=False)
            if df_filtered.is_empty():
                print("⚠️ No data found for given filters.")
                return
            df_filtered = df_filtered.with_columns([
                pl.col("Estimate").cast(pl.Float64)
            ])
        else:
            q = filtered_for_medical_cost.filter(
                (pl.col("Year") <= year) &
                (pl.col("Ownership") == ownership) &
                (pl.col("Provision") == provision)
            )
            df_filtered = collect_streaming_safe(q, streaming=True)

        if df_filtered.is_empty():
            print("⚠️ No matching data.")
            return

        agg = (
            df_filtered
            .group_by("Industry")
            .agg(pl.col("Estimate").mean().alias("Estimate"))
            .sort("Estimate", descending=False)
        )

        elapsed = time.perf_counter() - start_time
        mem_after = psutil.Process(os.getpid()).memory_info().rss / (1024**2)
        print(f"⏱️ Runtime: {elapsed:.2f}s | 💾 Δ{mem_after - mem_before:.2f} MB")

        fig = px.bar(
            agg.to_pandas(),
            x="Estimate", y="Industry", orientation="h",
            title=f"Average Estimate for {provision} ({ownership}) — Up to {year}",
            labels={"Estimate": "Average Estimate", "Industry": "Industry"},
            height=600
        )
        fig.update_layout(
            title={"text": "<br>".join(textwrap.wrap(fig.layout.title.text, width=70)),
                   "x": 0.5, "xanchor": "center", "yanchor": "top"},
            margin=dict(l=80, r=40, t=80, b=60)
        )
        fig.show()


# ==================================================
# ✅ STEP 4: Link Widgets & Display
# ==================================================
interactive_plot = widgets.interactive_output(
    update_plot, {"year": w_year_slider, "ownership": w_ownership_dropdown, "provision": w_provision_dropdown}
)

display(
    widgets.VBox([
        widgets.HBox([w_year_slider, w_ownership_dropdown]),
        w_provision_dropdown,
        output_widget
    ])
)


🔍 Backend mode: Aiven PostgreSQL
📡 Fetching metadata from Aiven (light queries)...


# MEDICAL PARTICIPATION AND TAKEUPS

In [226]:
# ==================================================
# ✅ MEDICAL CARE PARTICIPATION / TAKE-UP DASHBOARD — Hybrid (Aiven + Local)
# ==================================================
import polars as pl
import plotly.express as px
import textwrap
import ipywidgets as widgets
from IPython.display import display, clear_output
import psutil, time, os
import plotly.io as pio

# --- Helper: Safe streaming collector ---
def collect_streaming_safe(lf: pl.LazyFrame, streaming: bool = True) -> pl.DataFrame:
    """Collect a lazy Polars frame safely using streaming mode when available."""
    if not streaming:
        return lf.collect()
    try:
        return lf.collect(engine="streaming")
    except TypeError:
        return lf.collect(streaming=True)

# --- Detect backend ---
is_aiven = "conn" in globals() and conn is not None
print(f"🔍 Backend mode: {'Aiven PostgreSQL' if is_aiven else 'Local Parquet/LazyFrame'}")

# ==================================================
# ✅ STEP 1: Prepare Data
# ==================================================
if not is_aiven:
    # --- Local LazyFrame filtering ---
    filtered_for_rates = df_use.filter(
        (pl.col("Provision").str.contains("Medical care:", literal=True)) &
        (pl.col("Datatype").is_in(["Participation rate", "Take-up rate"]))
    )

    # --- Metadata collections ---
    unique_medical_provisions_rates = (
        filtered_for_rates.select("Provision").unique().collect()
        .to_series().drop_nulls().to_list()
    )
    ownership_options_rates = (
        filtered_for_rates.select("Ownership").unique().collect()
        .to_series().drop_nulls().to_list()
    )
    available_datatypes_rates = (
        filtered_for_rates.select("Datatype").unique().collect()
        .to_series().drop_nulls().to_list()
    )
    available_years_rates = (
        filtered_for_rates.select("Year").unique().collect()
        .to_series().drop_nulls().to_list()
    )

else:
    # --- Aiven SQL metadata queries ---
    print("📡 Fetching metadata from Aiven (lightweight queries)...")

    ownership_options_rates = (
        pl.read_database('SELECT DISTINCT "Ownership" FROM employee_benefits', conn)
        .to_series().drop_nulls().to_list()
    )
    unique_medical_provisions_rates = (
        pl.read_database(
            """SELECT DISTINCT "Provision"
               FROM employee_benefits
               WHERE "Provision" ILIKE 'Medical care:%'""",
            conn,
        ).to_series().drop_nulls().to_list()
    )
    available_datatypes_rates = (
        pl.read_database(
            """SELECT DISTINCT "Datatype"
               FROM employee_benefits
               WHERE "Datatype" IN ('Participation rate', 'Take-up rate')""",
            conn,
        ).to_series().drop_nulls().to_list()
    )
    available_years_rates = (
        pl.read_database('SELECT DISTINCT "Year" FROM employee_benefits', conn)
        .to_series().drop_nulls().to_list()
    )

# --- Year range setup ---
available_years_rates = sorted(
    int(y) for y in available_years_rates if isinstance(y, (int, float))
)
min_year_rates = available_years_rates[0] if available_years_rates else 2010
max_year_rates = available_years_rates[-1] if available_years_rates else 2025

# ==================================================
# ✅ STEP 2: Widgets
# ==================================================
w_year_slider_rates = widgets.IntSlider(
    value=max_year_rates,
    min=min_year_rates,
    max=max_year_rates,
    step=1,
    description="Year (Up to):",
    continuous_update=False,
)
w_ownership_dropdown_rates = widgets.SelectMultiple(
    options=["All"] + ownership_options_rates,
    value=("All",),
    description="Ownership:",
    layout=widgets.Layout(width="400px", height="100px"),
)
w_provision_dropdown_rates = widgets.SelectMultiple(
    options=["All"] + unique_medical_provisions_rates,
    value=("All",),
    description="Provision(s):",
    layout=widgets.Layout(width="500px", height="150px"),
)
w_datatype_dropdown_rates = widgets.SelectMultiple(
    options=["All"] + available_datatypes_rates,
    value=("All",),
    description="Datatype(s):",
    layout=widgets.Layout(width="400px", height="100px"),
)
output_widget_rates = widgets.Output()

# ==================================================
# ✅ STEP 3: Update Function — Dual Backend
# ==================================================
def update_rates_plot(year, ownership, provision, datatype):
    with output_widget_rates:
        clear_output(wait=True)
        print(f"📈 Filtering up to {year} | Ownership: {ownership} | Provision(s): {provision} | Datatype(s): {datatype}")

        start_time = time.perf_counter()
        mem_before = psutil.Process(os.getpid()).memory_info().rss / (1024**2)

        if is_aiven:
            # --- Build SQL dynamically (safe string construction) ---
            where = [f'"Year" <= {year}']

            if "All" not in ownership:
                safe_own = ", ".join([f"'{o}'" for o in ownership])
                where.append(f'"Ownership" IN ({safe_own})')

            if "All" not in provision:
                prov_conds = " OR ".join(
                    [f"LOWER(\"Provision\") LIKE '%{p.lower()}%'" for p in provision]
                )
                where.append(f"({prov_conds})")

            if "All" not in datatype:
                safe_dtype = ", ".join([f"'{d}'" for d in datatype])
                where.append(f'"Datatype" IN ({safe_dtype})')
            else:
                where.append('"Datatype" IN (\'Participation rate\', \'Take-up rate\')')

            sql = f"""
                SELECT "Year", "Datatype", "Estimate"
                FROM employee_benefits
                WHERE {' AND '.join(where)}
                  AND "Provision" ILIKE 'Medical care:%'
                  AND "Estimate" NOT IN ('-', '', 'N/A')
            """

            # --- Query Aiven safely ---
            df_filtered = safe_query(sql, conn, verbose=False)
            if df_filtered.is_empty():
                print("⚠️ No data found for selected filters.")
                return

            df_filtered = df_filtered.with_columns(pl.col("Estimate").cast(pl.Float64))

        else:
            # --- Local Polars path ---
            ownership_filter = (
                pl.col("Ownership").is_in(ownership)
                if "All" not in ownership
                else pl.lit(True)
            )
            provision_filter = (
                pl.col("Provision").is_in(provision)
                if "All" not in provision
                else pl.col("Provision").str.contains("Medical care:", literal=True)
            )
            datatype_filter = (
                pl.col("Datatype").is_in(datatype)
                if "All" not in datatype
                else pl.col("Datatype").is_in(["Participation rate", "Take-up rate"])
            )

            q = filtered_for_rates.filter(
                (pl.col("Year") <= year)
                & ownership_filter
                & provision_filter
                & datatype_filter
            )
            df_filtered = collect_streaming_safe(q, streaming=True)

        if df_filtered.is_empty():
            print(f"⚠️ No data found up to {year}.")
            return

        # --- Aggregate & plot ---
        agg = (
            df_filtered
            .group_by(["Year", "Datatype"])
            .agg(pl.col("Estimate").mean().alias("Estimate"))
            .sort("Year")
        )

        elapsed = time.perf_counter() - start_time
        mem_after = psutil.Process(os.getpid()).memory_info().rss / (1024**2)
        print(f"⏱️ Runtime: {elapsed:.2f}s | 💾 Δ{mem_after - mem_before:.2f} MB")

        fig = px.line(
            agg.to_pandas(),
            x="Year",
            y="Estimate",
            color="Datatype",
            markers=True,
            title=f"Average Participation / Take-up Rate (Medical Care) — Up to {year}",
        )
        fig.update_layout(
            title={
                "text": "<br>".join(textwrap.wrap(fig.layout.title.text, width=80)),
                "x": 0.5,
                "xanchor": "center",
                "yanchor": "top",
            },
            yaxis_title="Average Estimate (%)",
            height=600,
        )
        fig.show()

# ==================================================
# ✅ STEP 4: Connect Widgets & Display
# ==================================================
interactive_plot_rates = widgets.interactive_output(
    update_rates_plot,
    {
        "year": w_year_slider_rates,
        "ownership": w_ownership_dropdown_rates,
        "provision": w_provision_dropdown_rates,
        "datatype": w_datatype_dropdown_rates,
    },
)

display(
    widgets.VBox(
        [
            widgets.HBox([w_year_slider_rates, w_ownership_dropdown_rates]),
            widgets.HBox([w_provision_dropdown_rates, w_datatype_dropdown_rates]),
            output_widget_rates,
        ]
    )
)


🔍 Backend mode: Aiven PostgreSQL
📡 Fetching metadata from Aiven (light queries)...


# Other benefits analysis



In [227]:
benefit_keywords = [
    "dental", "vision", "life insurance", "long-term disability",
    "short-term disability", "outpatient", "paid sick leave",
    "medical", "hospital", "physician", "prescription", "health"
]

# Benefit Analysis by Keyword (COST DATA)

In [228]:
# ==================================================
# ✅ AVERAGE ESTIMATE BY INDUSTRY — KEYWORD DASHBOARD (Hybrid + Detailed Memory Tracking)
# ==================================================
import polars as pl
import plotly.express as px
import textwrap
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import psutil, time, os, gc
import plotly.io as pio

# --- Detect backend ---
is_aiven = "conn" in globals() and conn is not None
print(f"🔍 Backend mode: {'Aiven PostgreSQL' if is_aiven else 'Local Parquet/LazyFrame'}")


# ==================================================
# ✅ Helper: Safe Collector
# ==================================================
def collect_streaming_safe(lf: pl.LazyFrame, streaming: bool = True) -> pl.DataFrame:
    if not streaming:
        return lf.collect()
    try:
        return lf.collect(engine="streaming")
    except TypeError:
        return lf.collect(streaming=True)


# ==================================================
# ✅ Helper: Memory Stats Printer
# ==================================================
def memory_report(stage_label: str, mem_before=None):
    """Returns detailed process memory info."""
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()

    rss = mem_info.rss / (1024 ** 2)  # Resident memory (MB)
    vms = mem_info.vms / (1024 ** 2)  # Virtual memory (MB)
    data = getattr(mem_info, 'data', 0) / (1024 ** 2)

    if mem_before:
        delta_rss = rss - mem_before["rss"]
        delta_vms = vms - mem_before["vms"]
        delta_data = data - mem_before["data"]
        print(f"📈 {stage_label} Memory Δ — RSS: {delta_rss:+.2f} MB | VMS: {delta_vms:+.2f} MB | Data: {delta_data:+.2f} MB")
    else:
        print(f"💾 {stage_label} Memory — RSS: {rss:.2f} MB | VMS: {vms:.2f} MB | Data: {data:.2f} MB")

    return {"rss": rss, "vms": vms, "data": data}


# ==================================================
# ✅ STEP 1: Setup Keyword List
# ==================================================
benefit_keywords = [
    "dental", "vision", "life insurance", "long-term disability",
    "short-term disability", "outpatient", "paid sick leave",
    "medical", "hospital", "physician", "prescription", "health"
]


# ==================================================
# ✅ STEP 2: Prepare Data Sources
# ==================================================
if not is_aiven:
    # --- Local Parquet/LazyFrame ---
    filtered_for_average = df_use.filter(pl.col("Datatype") == "Average")

    all_unique_provisions_average = (
        filtered_for_average.select("Provision").unique().collect().to_series().drop_nulls().to_list()
    )
    ownership_options = (
        df_use.select("Ownership").unique().collect().to_series().drop_nulls().to_list()
    )
    available_years = (
        df_use.select("Year").unique().collect().to_series().drop_nulls().to_list()
    )

else:
    # --- Aiven SQL Mode (metadata fetch only) ---
    print("📡 Fetching metadata from Aiven...")
    all_unique_provisions_average = (
        pl.read_database(
            """SELECT DISTINCT "Provision" FROM employee_benefits WHERE "Datatype" = 'Average'""",
            conn
        ).to_series().drop_nulls().to_list()
    )

    ownership_options = (
        pl.read_database(
            """SELECT DISTINCT "Ownership" FROM employee_benefits""", conn
        ).to_series().drop_nulls().to_list()
    )

    available_years = (
        pl.read_database(
            """SELECT DISTINCT "Year" FROM employee_benefits""", conn
        ).to_series().drop_nulls().to_list()
    )

# --- Sanitize year range ---
available_years = sorted([int(y) for y in available_years if isinstance(y, (int, float))])
min_year, max_year = (available_years[0], available_years[-1]) if available_years else (2010, 2025)


# ==================================================
# ✅ STEP 3: Define Widgets
# ==================================================
w_year_slider_bar = widgets.IntSlider(
    value=max_year, min=min_year, max=max_year, step=1,
    description="Year (Up to):", continuous_update=False, layout=widgets.Layout(width="400px")
)

w_ownership_dropdown_bar = widgets.SelectMultiple(
    options=["All"] + ownership_options,
    value=("All",), description="Ownership:",
    layout=widgets.Layout(width="400px", height="100px")
)

w_provision_dropdown_bar = widgets.SelectMultiple(
    options=["All"], value=("All",), description="Provision(s):",
    layout=widgets.Layout(width="700px", height="150px")
)

w_benefit_keywords_bar = widgets.SelectMultiple(
    options=["All"] + benefit_keywords,
    value=("health",), description="Benefit Keywords:",
    layout=widgets.Layout(width="400px", height="100px")
)

output_widget_bar = widgets.Output()


# ==================================================
# ✅ STEP 4: Dynamic Provision Filtering
# ==================================================
def update_provision_options_bar(change):
    selected_keywords = w_benefit_keywords_bar.value

    if not selected_keywords or "All" in selected_keywords:
        filtered_provisions = all_unique_provisions_average
    else:
        if not is_aiven:
            keyword_filter = pl.col("Provision").str.to_lowercase().str.contains_any(
                [k.lower() for k in selected_keywords]
            )
            filtered_provisions = (
                filtered_for_average.filter(keyword_filter)
                .select("Provision").unique().collect().to_series().drop_nulls().to_list()
            )
        else:
            like_conditions = " OR ".join(
                [f"LOWER(\"Provision\") LIKE '%{k.lower()}%'" for k in selected_keywords]
            )
            sql = f'SELECT DISTINCT "Provision" FROM employee_benefits WHERE {like_conditions}'
            filtered_provisions = (
                pl.read_database(sql, conn).to_series().drop_nulls().to_list()
            )

    w_provision_dropdown_bar.options = ["All"] + sorted(filtered_provisions)
    if w_provision_dropdown_bar.value and not all(
        v in w_provision_dropdown_bar.options for v in w_provision_dropdown_bar.value
    ):
        w_provision_dropdown_bar.value = ("All",)
    elif not w_provision_dropdown_bar.value and "All" in w_provision_dropdown_bar.options:
        w_provision_dropdown_bar.value = ("All",)

w_benefit_keywords_bar.observe(update_provision_options_bar, names="value")
update_provision_options_bar(None)


# ==================================================
# ✅ STEP 5: Plot Update Function (Dual Backend + Memory Tracking)
# ==================================================
def update_bar_plot(year, ownership, provision, benefit_keywords):
    with output_widget_bar:
        clear_output(wait=True)
        print(f"📊 Generating chart up to {year}")
        print(f"🏢 Ownership: {ownership}")
        print(f"🏷️ Keywords: {benefit_keywords}")
        print("--------------------------------------------------")

        gc.collect()
        mem_start = memory_report("Start")

        start_time = time.perf_counter()

        if is_aiven:
            # --- Build SQL dynamically ---
            where = [f'"Datatype" = \'Average\'', f'"Year" <= {year}']

            if "All" not in ownership and ownership:
                safe_own = [f"'{o}'" for o in ownership if o.strip()]
                if safe_own:
                    where.append(f'"Ownership" IN ({", ".join(safe_own)})')

            if "All" in provision or not provision:
                if not benefit_keywords or "All" in benefit_keywords:
                    pass
                else:
                    conds = [f"LOWER(\"Provision\") LIKE '%{k.lower()}%'" for k in benefit_keywords]
                    where.append("(" + " OR ".join(conds) + ")")
            else:
                safe_prov = [f"'{p}'" for p in provision if p.strip()]
                if safe_prov:
                    where.append(f'"Provision" IN ({", ".join(safe_prov)})')

            sql = f"""
                SELECT "Industry", AVG(CAST(NULLIF("Estimate", '-') AS DOUBLE PRECISION)) AS "Estimate"
                FROM employee_benefits
                WHERE {' AND '.join(where)}
                GROUP BY "Industry"
                ORDER BY "Estimate" DESC
            """
            df_filtered = safe_query(sql, conn, verbose=False)
        else:
            # --- Local Polars LazyFrame ---
            ownership_filter = (
                pl.col("Ownership").is_in(ownership) if "All" not in ownership else pl.lit(True)
            )
            if "All" in provision:
                if not benefit_keywords or "All" in benefit_keywords:
                    provision_filter = pl.lit(True)
                else:
                    provision_filter = pl.col("Provision").str.to_lowercase().str.contains_any(
                        [k.lower() for k in benefit_keywords]
                    )
            else:
                provision_filter = pl.col("Provision").is_in(provision)

            q = (
                df_use.lazy()
                .select(["Industry", "Estimate", "Year", "Ownership", "Provision", "Datatype"])
                .filter(
                    (pl.col("Year") <= year)
                    & ownership_filter
                    & provision_filter
                    & (pl.col("Datatype") == "Average")
                )
            )
            df_filtered = collect_streaming_safe(q, streaming=True)
            df_filtered = (
                df_filtered.group_by("Industry")
                .agg(pl.col("Estimate").mean().alias("Estimate"))
                .sort("Estimate", descending=False)
            )

        mid_mem = memory_report("Post-Query", mem_start)

        if df_filtered.is_empty():
            print("⚠️ No data found for filters.")
            memory_report("End", mid_mem)
            return

        elapsed = time.perf_counter() - start_time

        gc.collect()
        mem_end = memory_report("Post-Aggregation", mid_mem)

        print(f"⏱️ Runtime: {elapsed:.2f}s")
        print(f"💾 Final Memory — RSS: {mem_end['rss']:.2f} MB | Δ: {mem_end['rss'] - mem_start['rss']:+.2f} MB")
        print("--------------------------------------------------")

        # --- Plot ---
        fig = px.bar(
            df_filtered.to_pandas(),
            x="Estimate", y="Industry", orientation="h",
            title=f"Average Estimate (Keywords: {', '.join(benefit_keywords)}) — Up to {year}",
            labels={"Estimate": "Average Estimate", "Industry": "Industry"},
            height=600,
        )
        fig.update_layout(
            title={"text": "<br>".join(textwrap.wrap(fig.layout.title.text, width=60)),
                   "x": 0.5, "xanchor": "center", "yanchor": "top"},
            margin=dict(l=80, r=40, t=80, b=60),
        )
        fig.show()


# ==================================================
# ✅ STEP 6: Connect Widgets & Display
# ==================================================
interactive_bar_plot = widgets.interactive_output(
    update_bar_plot,
    {
        "year": w_year_slider_bar,
        "ownership": w_ownership_dropdown_bar,
        "provision": w_provision_dropdown_bar,
        "benefit_keywords": w_benefit_keywords_bar,
    }
)

display(HTML("<h3>📊 Average Estimate by Industry (Filtered by Keywords, with Memory Profiling)</h3>"))
display(
    widgets.VBox([
        widgets.HBox([w_year_slider_bar, w_ownership_dropdown_bar]),
        widgets.HBox([w_provision_dropdown_bar, w_benefit_keywords_bar]),
        output_widget_bar,
    ])
)


🔍 Backend mode: Aiven PostgreSQL
📡 Fetching metadata from Aiven...


# OTEHR BENEFITS PARTICIPATION AND TAKE UP RATE ANALYSIS

In [229]:
# ==================================================
# ✅ RATES (PARTICIPATION & TAKE-UP) DASHBOARD — AIVEN OPTIMIZED (SAFE VERSION)
# ==================================================
import os, time, psutil, textwrap
import polars as pl
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import plotly.io as pio

# ==================================================
# ✅ CONFIG — BACKEND CONNECTION
# ==================================================
assert "conn" in globals() and conn is not None, "❌ Aiven connection not found — please define `conn` first."
print("📡 Using Aiven PostgreSQL backend — optimized for low memory usage")

# ==================================================
# ✅ STEP 1: Keyword definitions
# ==================================================
benefit_keywords = [
    "dental", "vision", "life insurance", "long-term disability",
    "short-term disability", "outpatient", "paid sick leave",
    "medical", "hospital", "physician", "prescription", "health"
]

# ==================================================
# ✅ STEP 2: Metadata queries (lightweight SQL)
# ==================================================
def fetch_list(sql: str):
    """Run lightweight DISTINCT query and return list of values."""
    df = safe_query(sql, conn, verbose=False)
    return sorted(df.to_series().drop_nulls().to_list())

ownership_options = fetch_list('SELECT DISTINCT "Ownership" FROM employee_benefits')
available_datatypes = fetch_list("""
    SELECT DISTINCT "Datatype"
    FROM employee_benefits
    WHERE "Datatype" IN ('Participation rate', 'Take-up rate')
""")
available_years = fetch_list('SELECT DISTINCT "Year" FROM employee_benefits ORDER BY "Year" ASC')
all_unique_provisions_rates = fetch_list("""
    SELECT DISTINCT "Provision"
    FROM employee_benefits
    WHERE "Datatype" IN ('Participation rate', 'Take-up rate')
""")

min_year = int(min(available_years)) if available_years else 2010
max_year = int(max(available_years)) if available_years else 2025

# ==================================================
# ✅ STEP 3: Widget definitions
# ==================================================
w_year_slider_rates = widgets.IntSlider(
    value=max_year, min=min_year, max=max_year, step=1,
    description="Year (Up to):", continuous_update=False,
    layout=widgets.Layout(width="400px")
)

w_ownership_dropdown_rates = widgets.SelectMultiple(
    options=["All"] + ownership_options,
    value=("All",), description="Ownership:",
    layout=widgets.Layout(width="400px", height="100px")
)

w_provision_dropdown_rates = widgets.SelectMultiple(
    options=["All"], value=("All",),
    description="Provision(s):",
    layout=widgets.Layout(width="700px", height="150px")
)

w_datatype_dropdown_rates = widgets.SelectMultiple(
    options=["All"] + available_datatypes,
    value=("All",), description="Datatype(s):",
    layout=widgets.Layout(width="400px", height="100px")
)

w_benefit_keywords_rates = widgets.SelectMultiple(
    options=["All"] + benefit_keywords,
    value=("health",), description="Benefit Keywords:",
    layout=widgets.Layout(width="400px", height="100px")
)

output_widget_rates = widgets.Output()

# ==================================================
# ✅ STEP 4: Dynamic provision updates (SQL-based)
# ==================================================
def update_provision_options_rates(change):
    selected_keywords = w_benefit_keywords_rates.value

    if not selected_keywords or "All" in selected_keywords:
        provisions = all_unique_provisions_rates
    else:
        like_conditions = " OR ".join(
            [f"LOWER(\"Provision\") LIKE '%{k.lower()}%'" for k in selected_keywords]
        )
        sql = f'SELECT DISTINCT "Provision" FROM employee_benefits WHERE {like_conditions}'
        provisions = fetch_list(sql)

    w_provision_dropdown_rates.options = ["All"] + provisions

    # reset selection if invalid
    if w_provision_dropdown_rates.value and not all(
        v in w_provision_dropdown_rates.options for v in w_provision_dropdown_rates.value
    ):
        w_provision_dropdown_rates.value = ("All",)
    elif not w_provision_dropdown_rates.value and "All" in w_provision_dropdown_rates.options:
        w_provision_dropdown_rates.value = ("All",)

w_benefit_keywords_rates.observe(update_provision_options_rates, names="value")
update_provision_options_rates(None)

# ==================================================
# ✅ STEP 5: Memory diagnostic helper
# ==================================================
def mem_info():
    p = psutil.Process(os.getpid())
    mi = p.memory_info()
    return mi.rss / (1024**2), mi.vms / (1024**2)

# ==================================================
# ✅ STEP 6: Update plot (Aiven SQL-based, Safe f-strings)
# ==================================================
def update_rates_plot(year, ownership, provision, datatype, benefit_keywords):
    with output_widget_rates:
        clear_output(wait=True)
        print(f"📊 Generating chart up to {year}")
        print(f"🏢 Ownership: {ownership}")
        print(f"🏷️ Datatypes: {datatype}")
        print(f"🔑 Keywords: {benefit_keywords}")
        print("-" * 50)

        rss_start, vms_start = mem_info()
        print(f"💾 Start Memory — RSS: {rss_start:.2f} MB | VMS: {vms_start:.2f} MB")

        # --- Build SQL dynamically (Safe version) ---
        where = [f'"Year" <= {year}', '"Datatype" IN (\'Participation rate\', \'Take-up rate\')']

        # ✅ Safe ownership filter
        if "All" not in ownership:
            safe_own = ", ".join([f"'{o}'" for o in ownership])
            where.append(f'"Ownership" IN ({safe_own})')

        # ✅ Safe provision filter
        if "All" not in provision:
            safe_prov = ", ".join([f"'{p}'" for p in provision])
            where.append(f'"Provision" IN ({safe_prov})')
        elif not ("All" in benefit_keywords or not benefit_keywords):
            conds = [f"LOWER(\"Provision\") LIKE '%{k.lower()}%'" for k in benefit_keywords]
            where.append("(" + " OR ".join(conds) + ")")

        # ✅ Safe datatype filter
        if "All" not in datatype:
            safe_dtype = ", ".join([f"'{d}'" for d in datatype])
            where.append(f'"Datatype" IN ({safe_dtype})')

        # --- Final SQL ---
        sql = f"""
            SELECT "Year", "Datatype",
                   AVG(CAST(NULLIF("Estimate", '-') AS DOUBLE PRECISION)) AS "Estimate"
            FROM employee_benefits
            WHERE {' AND '.join(where)}
            GROUP BY "Year", "Datatype"
            ORDER BY "Year"
        """

        start_query = time.perf_counter()
        df = safe_query(sql, conn)
        end_query = time.perf_counter()

        rss_mid, vms_mid = mem_info()
        print(f"📈 Post-Query Memory — RSS: {rss_mid:.2f} MB | Δ {rss_mid - rss_start:+.2f} MB")
        print(f"⏱️ Query time: {end_query - start_query:.2f}s")

        if df.is_empty():
            print("⚠️ No data found for current filters.")
            return

        # --- Plotting ---
        title_kw = ", ".join(benefit_keywords) if benefit_keywords and "All" not in benefit_keywords else "All Benefits"
        title_own = ", ".join(ownership) if "All" not in ownership else "All Ownerships"
        title_prov = ", ".join(provision) if "All" not in provision else "All Provisions"
        title_datatype = ", ".join(datatype) if "All" not in datatype else "Participation & Take-up Rates"

        title = f"Average Estimate Trend — {title_datatype} ({title_kw}) — {title_own} — Up to {year}"

        fig = px.line(
            df.to_pandas(),
            x="Year", y="Estimate", color="Datatype",
            markers=True, title=title
        )
        fig.update_layout(
            title={"text": "<br>".join(textwrap.wrap(title, width=80)), "x": 0.5, "xanchor": "center"},
            yaxis_title="Average Estimate (%)",
            height=600,
            margin=dict(l=60, r=40, t=80, b=60),
        )
        fig.show()

        rss_end, vms_end = mem_info()
        print(f"💾 End Memory — RSS: {rss_end:.2f} MB | Δ {rss_end - rss_start:+.2f} MB")
        print("-" * 50)

# ==================================================
# ✅ STEP 7: Connect & Display
# ==================================================
interactive_rates_plot = widgets.interactive_output(
    update_rates_plot,
    {
        "year": w_year_slider_rates,
        "ownership": w_ownership_dropdown_rates,
        "provision": w_provision_dropdown_rates,
        "datatype": w_datatype_dropdown_rates,
        "benefit_keywords": w_benefit_keywords_rates,
    }
)

display(HTML("<h3>📈 Average Estimate Trend (Participation & Take-up Rates) — Aiven Optimized</h3>"))
display(
    widgets.VBox([
        widgets.HBox([w_year_slider_rates, w_ownership_dropdown_rates]),
        widgets.HBox([w_provision_dropdown_rates, w_datatype_dropdown_rates, w_benefit_keywords_rates]),
        output_widget_rates,
    ])
)


📡 Using Aiven PostgreSQL backend — optimized for low memory usage
